# overview

We start from the raw PDBbind dataset downloaded from http://www.pdbbind.org.cn/download.php

1. filter out those unable to process using RDKit.

2. Process the protein by only preserving the chains that with at least one atom within 10Å from any atom of the ligand.

3. Use p2rank to segment protein into blocks.

4. extract protein and ligand features.

5. construct the training and test dataset.


In [1]:
import numpy as np
# test = info.query("group == 'test'").reset_index(drop=True)
# test_pdb_list = info.query("group == 'test'").protein_name.unique()
pre = "/home/jovyan/frag_protein/fragmentation/pdb_data/all_pdbbind"
# we use the time-split defined in EquiBind paper.
# https://github.com/HannesStark/EquiBind/tree/main/data
valid = np.loadtxt("../equbind/timesplit_no_lig_overlap_val", dtype=str)
test = np.loadtxt("../equbind/timesplit_test", dtype=str)
p2rank = "bash /home/jovyan/frag_protein/p2rank_2.3/prank"

In [2]:
tankbind_src_folder_path = "../tankbind/"
import sys
sys.path.insert(0, tankbind_src_folder_path)

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# process the raw PDBbind dataset.

In [4]:
from utils import read_pdbbind_data

In [5]:
# raw PDBbind dataset could be downloaded from http://www.pdbbind.org.cn/download.php

df_pdb_id = pd.read_csv(f'{pre}/index/INDEX_general_PL_name.2020', sep="  ", comment='#', header=None, names=['pdb', 'year', 'uid', 'd', 'e','f','g','h','i','j','k','l','m','n','o'], engine='python')
df_pdb_id = df_pdb_id[['pdb','uid']]
data = read_pdbbind_data(f'{pre}/index/INDEX_general_PL_data.2020')
data = data.merge(df_pdb_id, on=['pdb'])


# ligand file should be readable by RDKit.

In [6]:
from feature_utils import read_mol

In [7]:
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
pdb_list = []
probem_list = []
for pdb in tqdm(data.pdb):
    sdf_fileName = f"{pre}/{pdb}/{pdb}_ligand.sdf"
    mol2_fileName = f"{pre}/{pdb}/{pdb}_ligand.mol2"
    mol, problem = read_mol(sdf_fileName, mol2_fileName)
    if problem:
        probem_list.append(pdb)
        continue
    if pdb=="2r1w":
        continue
    pdb_list.append(pdb)

100%|██████████| 19443/19443 [01:38<00:00, 198.10it/s]


In [8]:
data = data.query("pdb in @pdb_list").reset_index(drop=True)

In [9]:
data.shape

(19127, 7)

### for ease of RMSD evaluation later, we renumber the atom index to be consistent with the smiles

In [10]:
from feature_utils import write_renumbered_sdf
import os

In [11]:
toFolder = f"{pre}/renumber_atom_index_same_as_smiles"
os.system(f"mkdir -p {toFolder}")
for pdb in tqdm(pdb_list):
    sdf_fileName = f"{pre}/{pdb}/{pdb}_ligand.sdf"
    mol2_fileName = f"{pre}/{pdb}/{pdb}_ligand.mol2"
    toFile = f"{toFolder}/{pdb}.sdf"
    write_renumbered_sdf(toFile, sdf_fileName, mol2_fileName)


100%|██████████| 19127/19127 [04:31<00:00, 70.39it/s]


# process PDBbind proteins, removing extra chains, cutoff 10A

In [12]:
toFolder = f"{pre}/protein_remove_extra_chains_10A/"
os.system(f"mkdir -p {toFolder}")

0

In [13]:
input_ = []
cutoff = 10
for pdb in data.pdb.values:
    pdbFile = f"{pre}/{pdb}/{pdb}_protein.pdb"
    ligandFile = f"{pre}/renumber_atom_index_same_as_smiles/{pdb}.sdf"
    toFile = f"{toFolder}/{pdb}_protein.pdb"
    x = (pdbFile, ligandFile, cutoff, toFile)
    input_.append(x)

In [14]:
from feature_utils import select_chain_within_cutoff_to_ligand_v2

In [15]:
import mlcrate as mlc
import os
pool = mlc.SuperPool(64)
pool.pool.restart()
_ = pool.map(select_chain_within_cutoff_to_ligand_v2,input_)
pool.exit()

[mlcrate] 64 CPUs:  46%|████▌     | 8781/19127 [03:02<03:23, 50.84it/s][03:51:57] bond with order 0 found on line 59. This is not part of the MDL specification.
[03:51:57] bond with order 0 found on line 61. This is not part of the MDL specification.
[mlcrate] 64 CPUs: 100%|██████████| 19127/19127 [06:19<00:00, 50.41it/s]


In [16]:
# previously, I found that 2r1w has no chain near the ligand.
data = data.query("pdb != '2r1w'").reset_index(drop=True)

# p2rank segmentation

In [17]:
p2rank_prediction_folder = f"{pre}/p2rank_protein_remove_extra_chains_10A"
os.system(f"mkdir -p {p2rank_prediction_folder}")
ds = f"{p2rank_prediction_folder}/protein_list.ds"
with open(ds, "w") as out:
    for pdb in data.pdb.values:
        out.write(f"../protein_remove_extra_chains_10A/{pdb}_protein.pdb\n")

In [18]:
# # takes about 30 minutes.
cmd = f"{p2rank} predict {ds} -o {p2rank_prediction_folder}/p2rank -threads 16"
os.system(cmd)

----------------------------------------------------------------------------------------------
 P2Rank 2.3
----------------------------------------------------------------------------------------------

predicting pockets for proteins from dataset [protein_list.ds]
processing [3zzf_protein.pdb] (1/19127)
processing [3fqa_protein.pdb] (4/19127)
processing [1aw1_protein.pdb] (15/19127)
processing aw_protein.pdb] (16/19127)
processing [2w97_protein.pdb] (11/19127)
processing [2b1r_protein.pdb] (13/19127)
processing [2d2v_protein.pdb] (14/19127)
processing [6h9v_protein.pdb] (6/19127)
processing [1p0y_protein.pdb] (12/19127)
processing [3gww_protein.pdb] (2/19127)
processing [4obv_protein.pdb] (7/19127)
processing [1w8l_protein.pdb] (3/19127)
processing [1zsb_protein.pdb] (5/19127)
processing [1wkm_protein.pdb] (8/19127)
processing [4eu3_protein.pdb] (10/19127)
processing [3k41_protein.pdb] (9/19127)
processing [3fl9_protein.pdb] (17/19127)
processing [5eb2_protein.pdb] (18/19127)
processi

0

In [19]:
data.to_csv(f"{pre}/data.csv")

In [20]:
pdb_list = data.pdb.values

In [22]:
tankbind_data_path = f"{pre}/tankbind_data"
name_list = pdb_list
d_list = []

for name in tqdm(name_list):
    p2rankFile = f"{pre}/p2rank_protein_remove_extra_chains_10A/p2rank/{name}_protein.pdb_predictions.csv"
    d = pd.read_csv(p2rankFile)
    d.columns = d.columns.str.strip()
    d_list.append(d.assign(name=name))
d = pd.concat(d_list).reset_index(drop=True)
d.reset_index(drop=True).to_feather(f"{tankbind_data_path}/p2rank_result.feather")

100%|██████████| 19127/19127 [01:11<00:00, 269.35it/s]


In [23]:
d = pd.read_feather(f"{tankbind_data_path}/p2rank_result.feather")

In [24]:
pockets_dict = {}
for name in tqdm(name_list):
    pockets_dict[name] = d[d.name == name].reset_index(drop=True)

100%|██████████| 19127/19127 [02:29<00:00, 128.08it/s]


# protein feature

In [25]:
from feature_utils import get_protein_feature

In [26]:
input_ = []
protein_embedding_folder = f"{tankbind_data_path}/gvp_protein_embedding"
os.system(f"mkdir -p {protein_embedding_folder}")
for pdb in pdb_list:
    proteinFile = f"{pre}/protein_remove_extra_chains_10A/{pdb}_protein.pdb"
    toFile = f"{protein_embedding_folder}/{pdb}.pt"
    x = (pdb, proteinFile, toFile)
    input_.append(x)

In [27]:
from Bio.PDB import PDBParser
from feature_utils import get_clean_res_list
import torch
torch.set_num_threads(1)

def batch_run(x):
    protein_dict = {}
    pdb, proteinFile, toFile = x
    parser = PDBParser(QUIET=True)
    s = parser.get_structure(pdb, proteinFile)
    res_list = get_clean_res_list(s.get_residues(), verbose=False, ensure_ca_exist=True)
    protein_dict[pdb] = get_protein_feature(res_list)
    torch.save(protein_dict, toFile)

In [28]:
import mlcrate as mlc
import os
pool = mlc.SuperPool(64)
pool.pool.restart()
_ = pool.map(batch_run,input_)
pool.exit()

[mlcrate] 64 CPUs: 100%|██████████| 19127/19127 [04:38<00:00, 68.61it/s]


In [29]:
import torch
protein_dict = {}
for pdb in tqdm(pdb_list):
    protein_dict.update(torch.load(f"{protein_embedding_folder}/{pdb}.pt"))


100%|██████████| 19127/19127 [03:08<00:00, 101.73it/s]


# Compound Features

In [30]:
from feature_utils import extract_torchdrug_feature_from_mol
compound_dict = {}
skip_pdb_list = []
for pdb in tqdm(pdb_list):
    mol, _ = read_mol(f"{pre}/renumber_atom_index_same_as_smiles/{pdb}.sdf", None)
    # extract features from sdf.
    try:
        compound_dict[pdb] = extract_torchdrug_feature_from_mol(mol, has_LAS_mask=True)  # self-dock set has_LAS_mask to true
    except Exception as e:
        print(e)
        skip_pdb_list.append(pdb)
        print(pdb)

  1%|          | 235/19127 [00:03<03:58, 79.32it/s]/opt/conda/envs/tankbind_py38/lib/python3.8/site-packages/torchdrug/data/feature.py:37: UserWarning: Unknown value `As`
  warnings.warn("Unknown value `%s`" % x)
  3%|▎         | 502/19127 [00:06<03:28, 89.47it/s]/opt/conda/envs/tankbind_py38/lib/python3.8/site-packages/torchdrug/data/feature.py:37: UserWarning: Unknown value `Fe`
  warnings.warn("Unknown value `%s`" % x)
  3%|▎         | 522/19127 [00:07<03:27, 89.70it/s]/opt/conda/envs/tankbind_py38/lib/python3.8/site-packages/torchdrug/data/feature.py:37: UserWarning: Unknown value `Pt`
  warnings.warn("Unknown value `%s`" % x)
 16%|█▌        | 2981/19127 [00:41<04:03, 66.20it/s]/opt/conda/envs/tankbind_py38/lib/python3.8/site-packages/torchdrug/data/feature.py:37: UserWarning: Unknown value `V`
  warnings.warn("Unknown value `%s`" % x)
/opt/conda/envs/tankbind_py38/lib/python3.8/site-packages/torchdrug/data/feature.py:37: UserWarning: Unknown value `8`
  warnings.warn("Unknown valu


3kqs


 37%|███▋      | 6995/19127 [01:46<03:22, 59.90it/s]/opt/conda/envs/tankbind_py38/lib/python3.8/site-packages/torchdrug/data/feature.py:37: UserWarning: Unknown value `Rh`
  warnings.warn("Unknown value `%s`" % x)
 50%|█████     | 9581/19127 [02:31<02:31, 63.09it/s]/opt/conda/envs/tankbind_py38/lib/python3.8/site-packages/torchdrug/data/feature.py:37: UserWarning: Unknown value `Ru`
  warnings.warn("Unknown value `%s`" % x)
 56%|█████▌    | 10644/19127 [02:49<02:44, 51.62it/s]/opt/conda/envs/tankbind_py38/lib/python3.8/site-packages/torchdrug/data/feature.py:37: UserWarning: Unknown value `Sb`
  warnings.warn("Unknown value `%s`" % x)
 58%|█████▊    | 11121/19127 [02:57<02:09, 61.85it/s]/opt/conda/envs/tankbind_py38/lib/python3.8/site-packages/torchdrug/data/feature.py:37: UserWarning: Unknown value `Co`
  warnings.warn("Unknown value `%s`" % x)
 58%|█████▊    | 11154/19127 [02:58<02:23, 55.67it/s]/opt/conda/envs/tankbind_py38/lib/python3.8/site-packages/torchdrug/data/feature.py:37: U

In [31]:
torch.save(compound_dict, f"{tankbind_data_path}/compound_torchdrug_features.pt")

In [32]:
skip_pdb_list

['3kqs']

In [33]:
data = data.query("pdb not in @skip_pdb_list").reset_index(drop=True)

# construct dataset.

In [34]:

def assign_group(pdb, valid=valid, test=test):
    if pdb in valid:
        return 'valid'
    if pdb in test:
        return 'test'
    return 'train'

data['group'] = data.pdb.map(assign_group)

In [35]:
data.value_counts("group")

group
train    17795
valid      968
test       363
dtype: int64

In [36]:
data['name'] = data['pdb']

In [37]:
info = []
err_pdb_list=[]
for i, line in tqdm(data.iterrows(), total=data.shape[0]):
    pdb = line['pdb']
    uid = line['uid']
    # smiles = line['smiles']
    smiles = ""
    affinity = line['affinity']
    group = line['group']

    compound_name = line['name']
    protein_name = line['name']
    try:
        pocket = pockets_dict[pdb].head(10)
    except:
        err_pdb_list.append(pdb)
        continue
    pocket.columns = pocket.columns.str.strip()
    pocket_coms = pocket[['center_x', 'center_y', 'center_z']].values
    # native block.
    info.append([protein_name, compound_name, pdb, smiles, affinity, uid, None, True, False, group])
    # protein center as a block.
    protein_com = protein_dict[protein_name][0].numpy().mean(axis=0).astype(float).reshape(1, 3)
    info.append([protein_name, compound_name, pdb+"_c", smiles, affinity, uid, protein_com, False, False, group])
    
    for idx, pocket_line in pocket.iterrows():
        pdb_idx = f"{pdb}_{idx}"
        info.append([protein_name, compound_name, pdb_idx, smiles, affinity, uid, pocket_coms[idx].reshape(1, 3), False, False, group])
info = pd.DataFrame(info, columns=['protein_name', 'compound_name', 'pdb', 'smiles', 'affinity', 'uid', 'pocket_com', 
                                   'use_compound_com', 'use_whole_protein',
                                  'group'])
print(len(info))


100%|██████████| 19126/19126 [00:18<00:00, 1051.58it/s]


162036


In [38]:
len(err_pdb_list)

0

In [39]:
len(pockets_dict)

19127

In [62]:
info.shape

(2879, 15)

In [41]:
from data import TankBindDataSet
import os

In [42]:
toFilePre = f"{pre}/dataset"
os.system(f"mkdir -p {toFilePre}")
dataset = TankBindDataSet(toFilePre, data=info, protein_dict=protein_dict, compound_dict=compound_dict)

Processing...
Done!


['/home/jovyan/TankBind/fragmentation/pdb_data/all_pdbbind/dataset/processed/data.pt', '/home/jovyan/TankBind/fragmentation/pdb_data/all_pdbbind/dataset/processed/protein.pt', '/home/jovyan/TankBind/fragmentation/pdb_data/all_pdbbind/dataset/processed/compound.pt']


In [87]:
toFilePre = f"{pre}/dataset"
dataset = TankBindDataSet(toFilePre)


['/home/jovyan/TankBind/fragmentation/pdb_data/all_pdbbind/dataset/processed/data.pt', '/home/jovyan/TankBind/fragmentation/pdb_data/all_pdbbind/dataset/processed/protein.pt', '/home/jovyan/TankBind/fragmentation/pdb_data/all_pdbbind/dataset/processed/compound.pt']


In [68]:
info = torch.load(f"/home/jovyan/TankBind/fragmentation/pdb_data/all_pdbbind/dataset/processed/data.pt")
info.shape

(162036, 15)

In [93]:
dataset[0]

HeteroData(
  dis_map=[1872],
  node_xyz=[144, 3],
  coords=[13, 3],
  y=[1872],
  seq=[144],
  affinity=[1],
  compound_pair=[169, 16],
  pdb='3zzf',
  group='train',
  real_affinity_mask=[1],
  real_y_mask=[1872],
  is_equivalent_native_pocket=[1],
  equivalent_native_y_mask=[1872],
  protein={
    node_s=[144, 6],
    node_v=[144, 3, 3]
  },
  compound={ x=[13, 56] },
  (protein, p2p, protein)={
    edge_index=[2, 3679],
    edge_s=[3679, 32],
    edge_v=[3679, 1, 3]
  },
  (compound, c2c, compound)={
    edge_index=[2, 24],
    edge_weight=[24],
    edge_attr=[24, 19]
  }
)

In [44]:
t = []
data = dataset.data
pre_pdb = None
for i, line in tqdm(data.iterrows(), total=data.shape[0]):
    pdb = line['compound_name']
    d = dataset[i]
    p_length = d['node_xyz'].shape[0]
    c_length = d['coords'].shape[0]
    y_length = d['y'].shape[0]
    num_contact = (d.y > 0).sum()
    t.append([i, pdb, p_length, c_length, y_length, num_contact])



100%|██████████| 162036/162036 [09:32<00:00, 283.18it/s]


In [45]:
# data = data.drop(['p_length', 'c_length', 'y_length', 'num_contact'], axis=1)

In [46]:
t = pd.DataFrame(t, columns=['index', 'pdb' ,'p_length', 'c_length', 'y_length', 'num_contact'])
t['num_contact'] = t['num_contact'].apply(lambda x: x.item())

In [47]:
data = pd.concat([data, t[['p_length', 'c_length', 'y_length', 'num_contact']]], axis=1)

In [48]:
native_num_contact = data.query("use_compound_com").set_index("protein_name")['num_contact'].to_dict()
data['native_num_contact'] = data.protein_name.map(native_num_contact)
# data['fract_of_native_contact'] = data['num_contact'] / data['native_num_contact']

In [49]:
torch.save(data, f"{toFilePre}/processed/data.pt")

In [59]:
toFilePre

'/home/jovyan/TankBind/fragmentation/pdb_data/all_pdbbind/test_dataset'

In [ ]:
test = info.query("group == 'test'").reset_index(drop=True)
test_pdb_list = info.query("group == 'test'").protein_name.unique()

In [ ]:
test = info.query("group == 'test'").reset_index(drop=True)
test_pdb_list = info.query("group == 'test'").protein_name.unique()

In [ ]:
subset_protein_dict = {}
for pdb in tqdm(test_pdb_list):
    subset_protein_dict[pdb] = protein_dict[pdb]

In [ ]:
subset_compound_dict = {}
for pdb in tqdm(test_pdb_list):
    subset_compound_dict[pdb] = compound_dict[pdb]

In [ ]:
toFilePre = f"{pre}/test_dataset"
os.system(f"mkdir -p {toFilePre}")
dataset = TankBindDataSet(toFilePre, data=test, protein_dict=subset_protein_dict, compound_dict=subset_compound_dict)

In [ ]:
toFilePre

In [139]:
import torch
info = torch.load(f"{toFilePre}/processed/data.pt")

In [146]:
info.head()

,protein_name,compound_name,pdb,smiles,affinity,uid,pocket_com,use_compound_com,use_whole_protein,group,p_length,c_length,y_length,num_contact,native_num_contact,index
0,6h9v,6h9v,6h9v,,0.66,B5BTR7,None,True,False,test,88,12,1056,78,78,0
1,6h9v,6h9v,6h9v_c,,0.66,B5BTR7,"[[8.569244384765625, -19.848827362060547, 54.5...",False,False,test,194,12,2328,5,78,1
2,6h9v,6h9v,6h9v_0,,0.66,B5BTR7,"[[10.4502, -35.1582, 63.1358]]",False,False,test,150,12,1800,0,78,2
3,6h9v,6h9v,6h9v_1,,0.66,B5BTR7,"[[-1.7136, -18.0661, 43.1919]]",False,False,test,157,12,1884,78,78,3
4,6h9v,6h9v,6h9v_2,,0.66,B5BTR7,"[[18.2234, -7.3891, 59.1918]]",False,False,test,150,12,1800,0,78,4


In [143]:
info.shape

(2879, 16)

In [111]:
def is_ligand_pocket(pdb):
    if len(pdb) == 4:
        return True
    else:
        return False
    
info['is_ligand_pocket'] = info.pdb.apply(lambda x:is_ligand_pocket(x))
pdb_to_num_contact = info.query("is_ligand_pocket").set_index("pdb")['num_contact'].to_dict()
info['base_pdb'] = info.pdb.apply(lambda x: x.split("_")[0])
info['native_num_contact'] = info.base_pdb.apply(lambda x: pdb_to_num_contact[x])
info['cover_contact_ratio'] = info['num_contact'] / info['native_num_contact']
info['index'] = info.index
use_whole_protein_list = set(info.base_pdb.unique()) - set(info.query("not is_ligand_pocket").base_pdb)
# assume we don't know the true ligand binding site.


In [121]:
info

,protein_name,compound_name,pdb,smiles,affinity,uid,pocket_com,use_compound_com,use_whole_protein,group,p_length,c_length,y_length,num_contact,native_num_contact,is_ligand_pocket,base_pdb,cover_contact_ratio
0,6h9v,6h9v,6h9v,,0.66,B5BTR7,None,True,False,test,88,12,1056,78,78,True,6h9v,1.000000
1,6h9v,6h9v,6h9v_c,,0.66,B5BTR7,"[[8.569244384765625, -19.848827362060547, 54.5...",False,False,test,194,12,2328,5,78,False,6h9v,0.064103
2,6h9v,6h9v,6h9v_0,,0.66,B5BTR7,"[[10.4502, -35.1582, 63.1358]]",False,False,test,150,12,1800,0,78,False,6h9v,0.000000
3,6h9v,6h9v,6h9v_1,,0.66,B5BTR7,"[[-1.7136, -18.0661, 43.1919]]",False,False,test,157,12,1884,78,78,False,6h9v,1.000000
4,6h9v,6h9v,6h9v_2,,0.66,B5BTR7,"[[18.2234, -7.3891, 59.1918]]",False,False,test,150,12,1800,0,78,False,6h9v,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2874,6oxp,6oxp,6oxp_0,,11.00,Q8ULI9,"[[71.2014, 55.9, 16.6762]]",False,False,test,130,40,5200,391,391,False,6oxp,1.000000
2875,6oxq,6oxq,6oxq,,11.30,Q8ULI9,None,True,False,test,131,41,5371,396,396,True,6oxq,1.000000
2876,6oxq,6oxq,6oxq_c,,11.30,Q8ULI9,"[[71.19599151611328, 59.35736846923828, 12.545...",False,False,test,150,41,6150,396,396,False,6oxq,1.000000
2877,6oxq,6oxq,6oxq_0,,11.30,Q8ULI9,"[[71.0965, 56.1167, 16.6608]]",False,False,test,131,41,5371,396,396,False,6oxq,1.000000


In [144]:
info.to_csv(f"{toFilePre}/apr23_testset_pdbbind_gvp_pocket_radius20_info.csv")

In [145]:
a = pd.read_csv(f"{toFilePre}/apr23_testset_pdbbind_gvp_pocket_radius20_info.csv")
a

,Unnamed: 0,protein_name,compound_name,pdb,smiles,affinity,uid,pocket_com,use_compound_com,use_whole_protein,group,p_length,c_length,y_length,num_contact,native_num_contact,index
0,0,6h9v,6h9v,6h9v,NaN,0.66,B5BTR7,NaN,True,False,test,88,12,1056,78,78,0
1,1,6h9v,6h9v,6h9v_c,NaN,0.66,B5BTR7,[[ 8.56924438 -19.84882736 54.56742859]],False,False,test,194,12,2328,5,78,1
2,2,6h9v,6h9v,6h9v_0,NaN,0.66,B5BTR7,[[ 10.4502 -35.1582 63.1358]],False,False,test,150,12,1800,0,78,2
3,3,6h9v,6h9v,6h9v_1,NaN,0.66,B5BTR7,[[ -1.7136 -18.0661 43.1919]],False,False,test,157,12,1884,78,78,3
4,4,6h9v,6h9v,6h9v_2,NaN,0.66,B5BTR7,[[18.2234 -7.3891 59.1918]],False,False,test,150,12,1800,0,78,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2874,2874,6oxp,6oxp,6oxp_0,NaN,11.00,Q8ULI9,[[71.2014 55.9 16.6762]],False,False,test,130,40,5200,391,391,2874
2875,2875,6oxq,6oxq,6oxq,NaN,11.30,Q8ULI9,NaN,True,False,test,131,41,5371,396,396,2875
2876,2876,6oxq,6oxq,6oxq_c,NaN,11.30,Q8ULI9,[[71.19599152 59.35736847 12.5458231 ]],False,False,test,150,41,6150,396,396,2876
2877,2877,6oxq,6oxq,6oxq_0,NaN,11.30,Q8ULI9,[[71.0965 56.1167 16.6608]],False,False,test,131,41,5371,396,396,2877


In [101]:
toFilePre

'/home/jovyan/TankBind/fragmentation/pdb_data/all_pdbbind/test_dataset'

In [56]:
def canonical_smiles(smiles):
    return Chem.MolToSmiles(Chem.MolFromSmiles(smiles))